In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('../../../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import config
from datasets.datasethandler import DatasetHandler
datasetHandler = DatasetHandler()

# Tutorial: Hyperparameter Search

In [6]:
import torch
import pytorch_lightning as pl
import optuna
from optuna.integration import PyTorchLightningPruningCallback

from classification.models.M5 import M5PLModule
from classification.models.SpectrogramCNN import SpectrogramCNNPLModule
from classification.models.DeepRecursiveCNN import DeepRecursiveCNNPLModule
from classification.models.MelSpectrogramCNN_8K import MelSpectrogramCNN_8KPLModule

from classification.models.SpectrogramCNN_8K_Dataset2 import SpectrogramCNN_8K_Dataset2_PLModule
from classification.models.CRNN_8k_D2 import CRNN8k_D2_PLModule
from classification.trainer.HyperParamSearch import MetricsCallback, save_model

In [7]:
from pytorch_lightning.callbacks import Callback

class SaveBestCallback(Callback): 
    def __init__(self, model_name = "newest_model", add_v_number = True):
        super().__init__()
        self.model_name = model_name
        self.best_val_acc = None
        self.add_v_number = add_v_number

    def on_epoch_end(self, trainer, pl_module):
        if not self.best_val_acc or pl_module.val_results_history[-1]["val_acc"] > self.best_val_acc:
            print("new best val acc", pl_module.val_results_history[-1]["val_acc"])
            self.best_val_acc = pl_module.val_results_history[-1]["val_acc"]
            #save_path = self.model_name + (( "_v{}".format(trainer.logger.version) +  "_best.p") if self.add_v_number else "")
            save_path = self.model_name + str(pl_module.val_results_history[-1]["val_acc"]) + ".p"
            pl_module.save(save_path, overwrite_if_exists=True)
            print("Saved checkpoint at epoch {} at \"{}\"".format((trainer.current_epoch + 1), save_path))

cb = SaveBestCallback("optuna_crnn8kd2_", add_v_number = False)

### Objective

In [9]:
def objective(trial):
    metrics_callback = MetricsCallback()  

    # here we sample the hyper params, similar as in our old random search
    trial_hparams = {"batch_size": 64,
                     "learning_rate": trial.suggest_loguniform("learning_rate", 1e-6, 1e-1),
                     "p_dropout": trial.suggest_float("p_drop", 0, 1),
                     "lr_decay": trial.suggest_float("lr_decay", 0.7, 1),
                     "n_hidden": trial.suggest_int("n_hidden", 10, 1000),
                     "lstm_hidden_size": trial.suggest_int("n_hidden", 10, 1000),
                     "weight_decay": trial.suggest_float("weight_decay", 1e-6, 1e-1)
                    }    

    model = CRNN8k_D2_PLModule(trial_hparams)
    
    # create a trainer
    trainer = pl.Trainer(
        logger=pl.loggers.TensorBoardLogger(config.LOG_DIR, name=type(model.model).__name__),
        max_epochs=15,                                                               
        gpus=1 if torch.cuda.is_available() else None,
        callbacks=[metrics_callback, cb],                                                   # save latest accuracy
        early_stop_callback=PyTorchLightningPruningCallback(trial, monitor="validation_acc"), # early stopping
    )
    
    datasetHandler.load(model, 'training', dataset_id=config.DATASET_CONTROL)
    datasetHandler.load(model, 'validation', dataset_id=config.DATASET_CONTROL)
    trainer.fit(model)

    # save model
    model.save("saved_modelsq/"+'{}.p'.format(trial.number))

    # return validation accuracy from latest model, as that's what we want to minimize by our hyper param search
    return metrics_callback.metrics[-1]["validation_acc"]

pruner = optuna.pruners.MedianPruner()
study = optuna.create_study(direction="maximize", pruner=pruner, study_name='CRNN_8K_D2', storage='sqlite:///CRNN_8K_D2_study.db', load_if_exists=True)

[I 2020-07-14 23:36:06,119] Using an existing study with name 'CRNN_8K_D2' instead of creating a new one.


### Run Search

In [13]:
study.optimize(objective, n_trials=2000, timeout=1.5*21600) #9h

GPU available: True, used: True
No environment variable for node rank defined. Set as 0.
CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

   | Name                      | Type                  | Params
----------------------------------------------------------------
0  | model                     | CRNN8k_D2             | 1 M   
1  | model.spec                | MelspectrogramStretch | 0     
2  | model.spec.spectrogram    | Spectrogram           | 0     
3  | model.spec.mel_scale      | MelScale              | 0     
4  | model.spec.stft           | Spectrogram           | 0     
5  | model.spec.random_stretch | RandomTimeStretch     | 0     
6  | model.spec.complex_norm   | ComplexNorm           | 0     
7  | model.spec.norm           | SpecNormalization     | 0     
8  | model.convs               | Sequential            | 213 K 
9  | model.convs.0             | Conv2d                | 320   
10 | model.convs.1             | BatchNorm2d           | 64    
11 | model.convs.2       

Val-Acc=0.03793716656787196


Val-Acc=0.4896265560165975
Train-Acc=0.5191699604743083
new best val acc 0.4896265560165975
Saved model to "optuna_crnn8kd2_0.4896265560165975.p"
Saved checkpoint at epoch 1 at "optuna_crnn8kd2_0.4896265560165975.p"


Val-Acc=0.4896265560165975
Train-Acc=0.525098814229249


Val-Acc=0.4896265560165975
Train-Acc=0.5227272727272727


...


Saved model to "saved_modelsq/93.p"


Exception ignored in: <function tqdm.__del__ at 0x7f637d070430>
Traceback (most recent call last):
  File "/nfs/students/summer-term-2020/project-4/yan/.conda/envs/cnn_project/lib/python3.8/site-packages/tqdm/std.py", line 1085, in __del__
    def __del__(self):
KeyboardInterrupt: 
[I 2020-07-15 01:03:07,563] Finished trial#93 with value: 0.8565500889152341 with parameters: {'learning_rate': 0.0007355082893104371, 'p_drop': 0.15782142156207, 'lr_decay': 0.790924682678241, 'n_hidden': 99, 'weight_decay': 0.003289985446402963}. Best is trial#44 with value: 0.8944872554831061.
GPU available: True, used: True
No environment variable for node rank defined. Set as 0.
CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

   | Name                      | Type                  | Params
----------------------------------------------------------------
0  | model                     | CRNN8k_D2             | 5 M   
1  | model.spec                | MelspectrogramStretch | 0     
2  | model.spec.spec

Val-Acc=0.03793716656787196


Detected KeyboardInterrupt, attempting graceful shutdown...


KeyboardInterrupt: 

In [12]:
#!mkdir saved_modelsq

In [14]:
print("Number of finished trials: {}".format(len(study.trials)))
print("Best trial:")
best_trial = study.best_trial
print("  Value: {}".format(best_trial.value))
print("  Params: ")
for key, value in best_trial.params.items():
    print("    {}: {}".format(key, value))

Number of finished trials: 95
Best trial:
  Value: 0.8944872554831061
  Params: 
    learning_rate: 0.0007190652771348608
    lr_decay: 0.7626277681903325
    n_hidden: 996
    p_drop: 0.15570993075113893
    weight_decay: 0.017084714300831555
